In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import regex as re

In [2]:
imonitor = pd.read_csv('data/imonitor_1703.csv')
imonitor.head()

C:\Users\MOgamba\AppData\Local\Temp\ipykernel_24456\2747984450.py:1: DtypeWarning: Columns (1,4,11,15,24,25,42,56,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  imonitor = pd.read_csv('data/imonitor_1703.csv')


,Survey ID,Created Date,Facility name and MFL Code if applicable,Facility ownership,Please specify,County,What is your month; and year of birth,How do you consider yourself?,What is the highest level of education you completed?,Please specify.1,...,how long do you wait on average to get a service; which service was that?,Do you consider the waiting time for lab test results long?,how long do you wait on average to get your lab test result?,Does the facility offer support groups?,Specify the support group you belong to,In your opinion are the services offered at this facility youth friendly?,What measures have been put in place to create GBV awareness and its harmful effects within the community?,Please Specify,PWD In your opinion are the services offered at this facility persons-with-disability friendly?,What are the top 1-3 things you don’t like about this facility with regards to care and treatment?
0,2390063,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1977-09-03,Male,Primary school,NaN,...,NaN,No,NaN,Yes,Adults,Yes,Presence of GBV Desk;,Chiefs office,Yes,NaN
1,2390062,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1972-08-12,Female,Secondary school,NaN,...,NaN,No,NaN,No,NaN,Yes,Presence of GBV Desk;,Chiefs office,NaN,NaN
2,2390061,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1984-08-31,Female,Primary school,NaN,...,NaN,Yes,2 hours,No,NaN,Yes,Presence of GBV Desk;,Chiefs office,Yes,NaN
3,2390060,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1977-05-07,Female,Primary school,NaN,...,NaN,No,NaN,No,NaN,Yes,Presence of GBV Desk;,Police station,NaN,NaN
4,2390059,04-Dec-23,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1987-06-13,Male,Vocational training or technician,NaN,...,1 hour,Yes,2 hours,Yes,Adults,Yes,Presence of GBV Desk;,Police station,NaN,NaN


In [3]:
imonitor.shape

(46549, 85)

In [4]:
# Find and drop columns that contain "Please specify" or "Please Specify"
cols_to_drop = [col for col in imonitor.columns if "Please specify" in col or "Please Specify" in col]

# Drop these columns from the DataFrame in a single operation
imonitor.drop(cols_to_drop, axis=1, inplace=True)

In [5]:
imonitor.shape

(46549, 69)

In [6]:
column_name_mapping = {
    "Survey ID": "SurveyID",
    "Organization name coordinating the feedback from the clients": "OrgFeedbackCoordinator",
    "Created Date": "FeedbackDate",
    "Facility name and MFL Code if applicable": "FacilityName",
    "Facility ownership": "FacilityOwnership",
    "County": "FacilityCounty",
    "What is your month; and year of birth": "BirthMonthYear",
    "How do you consider yourself? ": "SelfIdentity",
    "What is the highest level of education you completed?": "EducationLevel",
    "What is your current marital status? ": "MaritalStatus",
    "Which county do you currently live in?": "ResidenceCounty",
    "What are your sources of income?": "IncomeSources",
    "For how long have you been accessing services (based on the expected package of services) in this facility?": "ServiceAccessDuration",
    "Are you aware of the package of services that you are entitled to?": "ServicesAwareness",
    "According to you; which HIV related services are you likely to receive in this facility?": "ExpectedHIVServices",
    "Is there a service that you needed that was not provided?": "UnprovidedService",
    "Facility name no service": "UnprovidedServiceFacilityName",
    "For that service that was not provided; were you referred?": "ReferralForUnprovidedService",
    "If referred; did you receive the service where you were referred to?": "ReferralServiceReceived",
    "If Yes which Service/Test/Medicine ": "ReceivedServiceDetail",
    "On a scale of 1 to 5; how satisfied are you with the package of services received in this facility? If 1 is VERY UNSATISFIED and 5 is VERY SATISFIED.": "ServiceSatisfaction",
    "What did you like about the services you received?": "ServicesLiked",
    "What did you not like about the services you received?": "ServicesDisliked",
    "In your opinion what would you like to be improved?": "ImprovementSuggestions",
    "Do you face any challenges when accessing the services at the facility?": "AccessChallenges",
    "Common issues that can be added in the drop-down box": "CommonIssuesDropdown",
    "In your opinion what can be done to improve access to the services you seek at the facility?": "AccessImprovementSuggestions",
    "Was confidentiality considered while you were being served?": "Confidentiality",
    "Are there age-appropriate health services for specific groups?": "AgeAppropriateServices",
    "Does the facility allow you to share your concerns with the administration?": "ConcernsSharing",
    "Do you know your health-related rights as a client of this facility?": "RightsAwareness",
    "Have you ever been denied services at this facility?": "ServiceDenial",
    "Facility name denied service": "ServiceDenialFacilityName",
    "Why": "ServiceDenialSpecify",
    "Are you comfortable with getting services at this facility": "ComfortWithServices",
    "Have you ever been counseled? ": "CounselingReceived",
    "Did you identify any gaps in the facility when you tried to access the services": "IdentifiedGaps",
    "Service type": "ServiceGapsType",
    "Are the HIV testing services readily available when required? ": "HIVTestingAvailability",
    "Have you ever Interrupted your treatment?": "TreatmentInterruption",
    "Are the PMTCT services readily available when required?": "PMTCTServiceAvailability",
    "Were reasons provided as to why these services were not available?": "PMTCTServiceNonavailabilityReasons",
    "Are the HIV prevention; testing; treatment and care services adequate for KPs? ": "KPServiceAdequacy",
    "Were reasons provided as to why these services were not available?.1": "KPServiceNonavailabilityReasons",
    "What are the barriers to uptake of VMMC by males 25+years and above?": "VMMCBarriers",
    "What are some of the current site level practices that community members like and would love to maintain for KP/PP ?": "KPCommunityPreferredPractices",
    "What would you like this facility to change/do better?": "ChangeSuggestions",
    "Throughout your visit what did you find interesting/pleasing about this facility that should be emulated by other facilities?": "PositiveObservations",
    "What do you think can be improved": "GeneralImprovementSuggestions",
    "Anything else that you would like to mention?": "AdditionalComments",
    "What are the top 1-3 things you like about this facility with regards to care and treatment? ": "TopFacilityFeatures",
    "Facility Level": "FacilityLevel",
    "Facility Operation times": "OperationTimes",
    "Facility Operation Days ": "OperationDays",
    "What are your preferred days of visiting the facility": "PreferredVisitDays",
    "What are your preferred time of visiting the facility": "PreferredVisitTimes",
    "According to you, which HIV related service/tests/medicine are you likely to receive in this facility?": "ExpectedHIVServices2",
    "Is there a service/test/medicine that you needed that was not provided?": "UnprovidedService2",
    "which service/test/medicine?": "UnprovidedServiceDetail",
    "On a scale of 1-5; how clean do you find the facility?": "FacilityCleanliness",
    "How do you reach this facility?": "FacilityAccessMode",
    "How long does it take to reach this facility?": "FacilityAccessTime",
    "On a scale of 1-5; how accessible do you find this facility?": "FacilityAccessibility",
    "Do you consider the waiting time to be seen at this facility long?": "WaitingTimeOpinion",
    "how long do you wait on average to get a service; which service was that?": "AverageWaitingTime",
    "Do you consider the waiting time for lab test results long?": "LabResultsWaitingTimeOpinion",
    "how long do you wait on average to get your lab test result?": "AverageLabResultsWaitingTime",
    "Does the facility offer support groups?": "SupportGroupAvailability",
    "Specify the support group you belong to": "SpecifySupportGroup",
    "In your opinion are the services offered at this facility youth friendly?": "YouthFriendlyServices",
    "What measures have been put in place to create GBV awareness and its harmful effects within the community? ": "GBVAwarenessMeasures",
    "PWD In your opinion are the services offered at this facility persons-with-disability friendly?": "PWDFriendlyServicesOpinion",
    "What are the top 1-3 things you don’t like about this facility with regards to care and treatment?": "TopFacilityDislikes"
}

df = imonitor.rename(columns=column_name_mapping)

In [7]:
for column in df.columns:
    print(column)

SurveyID
FeedbackDate
FacilityName
FacilityOwnership
FacilityCounty
BirthMonthYear
SelfIdentity
EducationLevel
MaritalStatus
ResidenceCounty
IncomeSources
ServiceAccessDuration
ServicesAwareness
ExpectedHIVServices
UnprovidedService
Facility name
ReferralForUnprovidedService
ReferralServiceReceived
ReceivedServiceDetail
ServiceSatisfaction
ServicesLiked
ServicesDisliked
ImprovementSuggestions
AccessChallenges
CommonIssuesDropdown
AccessImprovementSuggestions
Confidentiality
AgeAppropriateServices
ConcernsSharing
RightsAwareness
ServiceDenial
ServiceDenialFacilityName
ServiceDenialSpecify
ComfortWithServices
CounselingReceived
IdentifiedGaps
ServiceGapsType
HIVTestingAvailability
TreatmentInterruption
PMTCTServiceAvailability
PMTCTServiceNonavailabilityReasons
KPServiceAdequacy
KPServiceNonavailabilityReasons
VMMCBarriers
KPCommunityPreferredPractices
ChangeSuggestions
PositiveObservations
GeneralImprovementSuggestions
AdditionalComments
TopFacilityFeatures
FacilityLevel
OperationTimes


In [8]:
columns_to_clean1 = [
    'WaitingTimeOpinion',
    'LabResultsWaitingTimeOpinion'
]

def replace_dont_know(df, column):
    df[column] = df[column].replace("Dont Know", "I don't know", regex=False)
    return df

for column in columns_to_clean1:
    df = replace_dont_know(df, column)

In [9]:
columns_to_clean2 = [
    'FacilityCleanliness',
    'FacilityAccessibility'
    ]

def replace_mixed_with_text(df, column_name):
    def replace_value(value):
        satisfaction_map = {
            1: 'Very Unsatisfied',
            2: 'Unsatisfied',
            3: 'Okay',
            4: 'Satisfied',
            5: 'Very Satisfied'
        }
        if isinstance(value, str) and value[0].isdigit():
            num = int(value[0])
        elif isinstance(value, int):
            num = value
        else:
            return value

        return satisfaction_map.get(num, value)

    df[column_name] = df[column_name].apply(replace_value)
    return df

for column in columns_to_clean2:
    df = replace_mixed_with_text(df, column)

In [10]:
def standardize_satisfaction(df, column_name):
    # Mapping for consolidating variations of satisfaction levels
    satisfaction_map = {
        '5': 'Very Satisfied',
        5.0: 'Very Satisfied',
        '4': 'Satisfied',
        4.0: 'Satisfied',
        '3': 'Okay',
        3.0: 'Okay',
        '2': 'Unsatisfied',
        2.0: 'Unsatisfied',
        '1': 'Very Unsatisfied',
        1.0: 'Very Unsatisfied',
        'Dissatisfied': 'Unsatisfied'
    }
    
    # Replace values based on the map
    df[column_name] = df[column_name].replace(satisfaction_map)
    return df

df = standardize_satisfaction(df, 'ServiceSatisfaction')


In [11]:
print(df['FacilityLevel'].value_counts())

FacilityLevel
4.0    4802
3.0    4515
2.0    2889
5.0    2240
1.0     556
6.0      14
Name: count, dtype: int64


In [12]:
def standardize_facility(df, column_name):
    # Mapping for consolidating variations of satisfaction levels
    satisfaction_map = {
        1.0: 'Community Health Unit',
        2.0: 'Dispensaries and Private Clinics',
        3.0: 'Health Centers',
        4.0: 'Sub-County Hospitals',
        5.0: 'County Referral Hospitals',
        6.0: 'National Referral Hospitals',
    }
    
    # Replace values based on the map
    df[column_name] = df[column_name].replace(satisfaction_map)
    return df

df = standardize_facility(df, 'FacilityLevel')

In [13]:
def replace_symbols_and_words(df, column_name):
    df[column_name] = df[column_name].str.replace('<', 'Less than', regex=False)
    df[column_name] = df[column_name].str.replace('>', 'More than', regex=False)
    df[column_name] = df[column_name].str.replace('minutes', 'mins', regex=False)
    return df

df = replace_symbols_and_words(df, 'FacilityAccessTime')

In [14]:
def replace_symbols_and_words(df, column_name):
    df[column_name] = df[column_name].str.replace('Less than 30mins', 'Less than 30 mins', regex=False)
    df[column_name] = df[column_name].str.replace('More than45 mins', 'More than 45 mins', regex=False)
    return df

df = replace_symbols_and_words(df, 'FacilityAccessTime')

In [15]:
def remove_trailing_semicolons(df, column_names):
    for column in column_names:
        if column in df.columns:
            df[column] = df[column].str.rstrip(';')
    return df

df = pd.DataFrame(df)

columns_to_clean = ['ExpectedHIVServices', 'OperationTimes', 'OperationDays', 'PreferredVisitDays', 'PreferredVisitTimes', 'GBVAwarenessMeasures']

df2 = remove_trailing_semicolons(df, columns_to_clean)

In [16]:
def calculate_age(birth_date_str):
    if isinstance(birth_date_str, str):
        # Parse the birth date string into a datetime object
        birth_date = datetime.strptime(birth_date_str, "%Y-%m-%d")
        
        # Get today's date
        today = datetime.today()
        
        # Calculate age
        age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
        
        return age
    else:
        # Return None or an appropriate value for missing or invalid input
        return None
    
df2['Age'] = df2['BirthMonthYear'].apply(calculate_age)

In [17]:
def remove_invalid_age_rows(df2, age_column):
    valid_age_condition = (df[age_column] >= 0) & (df[age_column] <= 100)
    
    # Keep only rows with valid age
    cleaned_df = df2[valid_age_condition].copy()
    
    return cleaned_df
df3 = remove_invalid_age_rows(df2, 'Age')

In [18]:
df3.dropna(subset=['BirthMonthYear'], inplace=True)

In [19]:
def convert_mixed_dates(date_column):
    """
    This function takes a Pandas Series of mixed dates and Excel serial dates and converts them to datetime objects.
    
    Parameters:
    date_column (pd.Series): A pandas Series with mixed date formats and serial dates.
    
    Returns:
    pd.Series: A pandas Series with all dates converted to datetime objects.
    """
    # Define the epoch start for Excel's serial date format
    excel_epoch = pd.Timestamp('1899-12-30')
    converted_dates = []

    for date in date_column:
        if isinstance(date, str) and re.match(r'^\d+(\.\d+)?$', date):
            # If it's a string that looks like a serial date, convert it
            serial_value = float(date)
            converted_date = excel_epoch + pd.to_timedelta(serial_value, unit='D')
        elif isinstance(date, (int, float)):
            # If it's a numeric type, assume it's a serial date
            converted_date = excel_epoch + pd.to_timedelta(date, unit='D')
        else:
            # Otherwise, try to parse it as a regular date
            converted_date = pd.to_datetime(date, errors='coerce')

        # Append the result, which will be NaT if parsing failed
        converted_dates.append(converted_date)

    return pd.Series(converted_dates)

convert_mixed_dates(df3['FeedbackDate'])


0       2023-12-04
1       2023-12-04
2       2023-12-04
3       2023-12-04
4       2023-12-04
           ...    
41837   2022-10-04
41838   2022-10-03
41839   2022-10-04
41840   2022-10-03
41841   2022-09-28
Length: 41842, dtype: datetime64[ns]

In [20]:
df3.to_csv('data/cleaned.csv', index=False)

In [21]:
columns_to_keep = [
    'SurveyID', 'FeedbackDate', 'FacilityCounty', 'FacilityLevel', 'FacilityOwnership', 'ServicesLiked', 'ServicesDisliked', 'ImprovementSuggestions',
    'AccessImprovementSuggestions', 'PositiveObservations',
    'GeneralImprovementSuggestions', 'AdditionalComments', 'TopFacilityFeatures', 'ServiceSatisfaction', 'TopFacilityDislikes'
]

df_selected = df3[columns_to_keep]

df_selected.to_csv('data/clm_open_ended.csv', index=False)

In [22]:
percent_empty = {}
for column in df3.columns:
    # Calculate the number of empty values per column
    num_empty = df3[column].isnull().sum()
    # Calculate the percentage of empty values
    percent_empty[column] = (num_empty / len(df3)) * 100
    if percent_empty[column] > 1:
        print(f"{column}: {percent_empty[column]:.2f}%")

FacilityName: 7.08%
ServicesAwareness: 1.10%
ExpectedHIVServices: 7.38%
UnprovidedService: 1.28%
Facility name: 99.02%
ReferralForUnprovidedService: 98.24%
ReferralServiceReceived: 99.03%
ReceivedServiceDetail: 99.82%
ServicesLiked: 1.42%
ServicesDisliked: 15.71%
ImprovementSuggestions: 8.65%
AccessChallenges: 1.10%
CommonIssuesDropdown: 82.68%
AccessImprovementSuggestions: 10.29%
Confidentiality: 1.90%
AgeAppropriateServices: 2.01%
ConcernsSharing: 1.97%
RightsAwareness: 1.25%
ServiceDenial: 1.32%
ServiceDenialFacilityName: 99.82%
ServiceDenialSpecify: 99.94%
ComfortWithServices: 1.35%
CounselingReceived: 1.25%
IdentifiedGaps: 1.52%
ServiceGapsType: 94.12%
HIVTestingAvailability: 99.12%
TreatmentInterruption: 95.90%
PMTCTServiceAvailability: 99.88%
PMTCTServiceNonavailabilityReasons: 100.00%
KPServiceAdequacy: 99.95%
KPServiceNonavailabilityReasons: 100.00%
VMMCBarriers: 99.96%
KPCommunityPreferredPractices: 99.76%
ChangeSuggestions: 99.78%
PositiveObservations: 2.82%
GeneralImproveme

In [23]:
missing_percentage = df3.isnull().mean() * 100

threshold = 80

columns_to_drop = missing_percentage[missing_percentage > threshold].index.tolist()

print("Columns to drop:", columns_to_drop)

print("Number of columns to drop:", len(columns_to_drop))

df3.drop(columns=columns_to_drop, axis=1, inplace=True)

print("DataFrame shape after dropping columns:", df3.shape)

Columns to drop: ['Facility name', 'ReferralForUnprovidedService', 'ReferralServiceReceived', 'ReceivedServiceDetail', 'CommonIssuesDropdown', 'ServiceDenialFacilityName', 'ServiceDenialSpecify', 'ServiceGapsType', 'HIVTestingAvailability', 'TreatmentInterruption', 'PMTCTServiceAvailability', 'PMTCTServiceNonavailabilityReasons', 'KPServiceAdequacy', 'KPServiceNonavailabilityReasons', 'VMMCBarriers', 'KPCommunityPreferredPractices', 'ChangeSuggestions', 'AverageWaitingTime', 'AverageLabResultsWaitingTime', 'SpecifySupportGroup']
Number of columns to drop: 20
DataFrame shape after dropping columns: (41842, 50)


In [24]:
threshold_percentage = 50

threshold = len(df3.columns) * (threshold_percentage / 100)

data = df3.dropna(thresh=threshold)

print("Original DataFrame shape:", df3.shape)
print("Cleaned DataFrame shape:", data.shape)

rows_dropped = df3.shape[0] - data.shape[0]
print("Rows dropped:", rows_dropped)

Original DataFrame shape: (41842, 50)
Cleaned DataFrame shape: (41384, 50)
Rows dropped: 458


In [25]:
columns_null = [
    'FacilityName',
    'ExpectedHIVServices',
    'FacilityLevel',
    'OperationTimes',
    'OperationDays',
    'PreferredVisitDays',
    'PreferredVisitTimes',
    'FacilityCleanliness',
    'FacilityAccessMode',
    'FacilityAccessTime',
    'FacilityAccessibility',
    'WaitingTimeOpinion',
    'AverageWaitingTime',
    'LabResultsWaitingTimeOpinion',
    'AverageLabResultsWaitingTime',
    'SupportGroupAvailability',
    'YouthFriendlyServices',
    'GBVAwarenessMeasures',
    'PWDFriendlyServicesOpinion',
    'FacilityOwnership',
    'FacilityCounty',
    'SelfIdentity',
    'EducationLevel',
    'MaritalStatus',
    'ResidenceCounty',
    'IncomeSources',
    'ServiceAccessDuration',
    'ServicesAwareness',
    'UnprovidedService',
    'ServiceSatisfaction',
    'AccessChallenges',
    'Confidentiality',
    'AgeAppropriateServices',
    'ConcernsSharing',
    'RightsAwareness',
    'ServiceDenial',
    'ComfortWithServices',
    'CounselingReceived',
    'IdentifiedGaps'
]

for column in columns_null:
    if column in data.columns:
        data.fillna({column:'Unknown'}, inplace=True)

C:\Users\MOgamba\AppData\Local\Temp\ipykernel_24456\3772976849.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna({column:'Unknown'}, inplace=True)


In [26]:
columns_list = {}
for column in data.columns:
    print(column)

SurveyID
FeedbackDate
FacilityName
FacilityOwnership
FacilityCounty
BirthMonthYear
SelfIdentity
EducationLevel
MaritalStatus
ResidenceCounty
IncomeSources
ServiceAccessDuration
ServicesAwareness
ExpectedHIVServices
UnprovidedService
ServiceSatisfaction
ServicesLiked
ServicesDisliked
ImprovementSuggestions
AccessChallenges
AccessImprovementSuggestions
Confidentiality
AgeAppropriateServices
ConcernsSharing
RightsAwareness
ServiceDenial
ComfortWithServices
CounselingReceived
IdentifiedGaps
PositiveObservations
GeneralImprovementSuggestions
AdditionalComments
TopFacilityFeatures
FacilityLevel
OperationTimes
OperationDays
PreferredVisitDays
PreferredVisitTimes
FacilityCleanliness
FacilityAccessMode
FacilityAccessTime
FacilityAccessibility
WaitingTimeOpinion
LabResultsWaitingTimeOpinion
SupportGroupAvailability
YouthFriendlyServices
GBVAwarenessMeasures
PWDFriendlyServicesOpinion
TopFacilityDislikes
Age


In [27]:
# General descriptive statistics
data.describe()

,SurveyID,Age
count,4.138400e+04,41384.000000
mean,9.985729e+05,41.095617
std,1.047462e+06,13.069621
min,1.791480e+05,0.000000
25%,1.925865e+05,32.000000
50%,2.045255e+05,40.000000
75%,2.341208e+06,49.000000
max,2.390063e+06,100.000000


In [28]:
# Descriptive statistics for categorical data
data.describe(include=['object'])

,FeedbackDate,FacilityName,FacilityOwnership,FacilityCounty,BirthMonthYear,SelfIdentity,EducationLevel,MaritalStatus,ResidenceCounty,IncomeSources,...,FacilityAccessMode,FacilityAccessTime,FacilityAccessibility,WaitingTimeOpinion,LabResultsWaitingTimeOpinion,SupportGroupAvailability,YouthFriendlyServices,GBVAwarenessMeasures,PWDFriendlyServicesOpinion,TopFacilityDislikes
count,41384,41384,41384,41384,41384,41384,41384,41384,41384,41384,...,41384,41384,41384,41384,41384,41384,41384,41384,41384,10977
unique,13656,1505,7,11,14179,8,9,10,13,31,...,5,4,6,4,4,3,4,34,3,1928
top,09-Oct-23,Unknown,GOK,Homabay,1961-02-17,Female,Primary school,Married,Homabay,Business;,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Nothing
freq,2072,2843,33697,15863,208,27999,20182,24536,15520,8803,...,26173,26187,26174,26229,26244,26290,27309,26703,31197,2657


In [29]:
fig = px.histogram(data, x='ServiceSatisfaction', title='Distribution of Service Satisfaction')
fig.update_layout(bargap=0.2)
fig.show()

In [30]:
# Age Distribution
fig = px.histogram(data, x='Age', nbins=30, title='Age Distribution')
fig.update_traces(opacity=0.75)
fig.show()

In [31]:
# Relationship between Age and ServiceSatisfaction
fig = px.box(data, x='ServiceSatisfaction', y='Age', points="all", title='Age vs. Service Satisfaction')
fig.show()